In [1]:
import numpy as np
import pandas as pd
import os
from basic_tools import *

# Individuals common in data

In [6]:
pheno_ind=pd.read_csv(pheno_all_file_path,sep='\t')["ID"]

In [10]:
_1000G_ind=pd.read_csv('data/genotype/0_1000G/KCHIP_HLA_6_28Mb_35Mb_20200113_QC.fam',sep=' ',header=None)[0]

In [12]:
bmarker_ind=pd.read_csv('data/genotype/3_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.fam',sep=' ',header=None)[0]

In [35]:
individual_common=pd.DataFrame(pd.Index(pheno_ind).intersection(pd.Index(_1000G_ind)).intersection(pd.Index(bmarker_ind))).sort_values(0)[[0,0]]
individual_common.to_csv(individual_common_path,sep='\t',index=None,header=None)

# Imputation(CookHLA) included in HATK

HATK returns `*.ped` and `*.alleles`

# NomenCleaner

# BmarkerGenerator

# Beagle phasing

# AA call

In [4]:
run_subprocess(' '.join(['Rscript AllCC_Get_Phased_AA_Calls.R',
                bmarker_phased_path,
                bmarker_plink_path+'.fam',
                bmarker_aa_path                        
                        ]),dry=True)

------RUN-----
Rscript AllCC_Get_Phased_AA_Calls.R data/genotype/3_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.beagle5.1.AGM.bgl.phased data/genotype/3_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.fam data/genotype/3_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.beagle5.1.AGM.bgl.phased.aa


# KCHIP + 1000G

## 1000G

In [42]:
run_subprocess('plink --vcf data/genotype/0_1000G/KCHIP_HLA_6_28Mb_35Mb_20200113_QC.recode.vcf.gz --make-bed --out data/genotype/0_1000G/KCHIP_HLA_6_28Mb_35Mb_20200113_QC'.format(individual_common_path),dry=True)

------RUN-----
plink --vcf data/genotype/0_1000G/KCHIP_HLA_6_28Mb_35Mb_20200113_QC.recode.vcf.gz --make-bed --out data/genotype/0_1000G/KCHIP_HLA_6_28Mb_35Mb_20200113_QC


In [43]:
run_subprocess('plink --bfile data/genotype/0_1000G/KCHIP_HLA_6_28Mb_35Mb_20200113_QC --keep {} --make-bed --out data/genotype/0_1000G/KCHIP_HLA_6_28Mb_35Mb_20200113_QC.newfilter'.format(individual_common_path),dry=True)

------RUN-----
plink --bfile data/genotype/0_1000G/KCHIP_HLA_6_28Mb_35Mb_20200113_QC --keep data/individual.tsv --make-bed --out data/genotype/0_1000G/KCHIP_HLA_6_28Mb_35Mb_20200113_QC_newfilter


1000G
MAF 0.01
Imputation quality score 0.8

hwe 1e-10

In [44]:
_1000G_bim=pd.read_csv('data/genotype/0_1000G/KCHIP_HLA_6_28Mb_35Mb_20200113_QC_newfilter'+'.bim',sep='\t',names=['CHR','SNP','CM','BP','A1','A2'])
_1000G_bim.shape

(63544, 6)

In [48]:
bmarker_bim=pd.read_csv('data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele'+'.bim',sep='\t',names=['CHR','SNP','CM','BP','A1','A2'])
bmarker_bim.shape

(12101, 6)

In [49]:
_1000G_bmarker=_1000G_bim.rename(columns=lambda x: str(x)+'_1000G').merge(right=bmarker_bim.rename(columns=lambda x: str(x)+'_bmarker'),how='inner',left_on='BP_1000G',right_on='BP_bmarker')

In [51]:
_1000G_bmarker['SNP_1000G'].to_csv('data/genotype/0_1000G/1000G_bmarker_overlap.tsv',index=None,header=None)

/home/ch6845/tools/miniconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [52]:
run_subprocess('plink --bfile data/genotype/0_1000G/KCHIP_HLA_6_28Mb_35Mb_20200113_QC.newfilter --exclude data/genotype/0_1000G/1000G_bmarker_overlap.tsv --make-bed --out data/genotype/0_1000G/KCHIP_HLA_6_28Mb_35Mb_20200113_QC.newfilter.nooverlap',dry=True)

------RUN-----
plink --bfile data/genotype/0_1000G/KCHIP_HLA_6_28Mb_35Mb_20200113_QC_newfilter --exclude data/genotype/0_1000G/1000G_bmarker_overlap.tsv --make-bed --out data/genotype/0_1000G/KCHIP_HLA_6_28Mb_35Mb_20200113_QC_newfilter_nooverlap


## KCHIP

In [115]:
command="plink --bfile data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele \
--keep {} \
--maf 0.01 \
--make-bed \
--out data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC".format(individual_common_path)
command

'plink --bfile data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele --keep data/individual.tsv --maf 0.01 --make-bed --out data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC'

In [116]:
command="grep -P '\tAA_' data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC.bim| awk '{{print $2}}' > data/genotype/3_KCHIP_bmarkerphased/aa_list.tsv"
command

"grep -P '\tAA_' data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC.bim| awk '{{print $2}}' > data/genotype/3_KCHIP_bmarkerphased/aa_list.tsv"

In [117]:
command="plink --bfile data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC \
--exclude data/genotype/3_KCHIP_bmarkerphased/aa_list.tsv \
--make-bed \
--out data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC.noaa"
command

'plink --bfile data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC --exclude data/genotype/3_KCHIP_bmarkerphased/aa_list.tsv --make-bed --out data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC.noaa'

In [7]:
before_maf=pd.read_csv('data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.bim',sep='\t',names=['CHR','SNP','CM','BP','A1','A2'])

In [8]:
after_maf=pd.read_csv('data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC.bim',sep='\t',names=['CHR','SNP','CM','BP','A1','A2'])

In [9]:
removed=pd.Index(before_maf.SNP).difference(after_maf.SNP)
removed=pd.Series(removed)
removed_aaformat=['_'.join(i.split('_')[:3]) if i[:3]=='AA_' else i for i in removed]
removed_aaformat_nodup=pd.Series(removed_aaformat)[~pd.Series(removed_aaformat).duplicated(keep=False)].values


In [10]:
len(removed),len(removed_aaformat_nodup)

(627, 608)

## iid matching

In [2]:
individual_common=pd.read_csv(individual_common_path,sep='\t',header=None)[0]

In [3]:
f_in=open("data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.beagle5.1.AGM.bgl.phased.aa",'r')
iid_df=pd.DataFrame(f_in.readline().strip().split(' ')[2:],columns=['iid']).reset_index()
f_in.close()

In [4]:
iid_df.shape[0]/2

125872.0

In [5]:
iid_df_merged=pd.DataFrame(individual_common).merge(right=iid_df,right_on='iid',left_on=0,how='inner')

In [6]:
assert (iid_df_merged.iid.unique()!=individual_common).sum()==0

In [9]:
iid_df

,index,iid
0,0,BHAN000001
1,1,BHAN000001
2,2,BHAN000002
3,3,BHAN000002
4,4,BHAN000003
...,...,...
251739,251739,BHAN125870
251740,251740,BHAN125871
251741,251741,BHAN125871
251742,251742,BHAN125872


In [8]:
iid_df_merged

,0,index,iid
0,BHAN000001,0,BHAN000001
1,BHAN000001,1,BHAN000001
2,BHAN000002,2,BHAN000002
3,BHAN000002,3,BHAN000002
4,BHAN000003,4,BHAN000003
...,...,...,...
251341,BHAN125870,251739,BHAN125870
251342,BHAN125871,251740,BHAN125871
251343,BHAN125871,251741,BHAN125871
251344,BHAN125872,251742,BHAN125872


In [7]:
iid_df['index']

0              0
1              1
2              2
3              3
4              4
           ...  
251739    251739
251740    251740
251741    251741
251742    251742
251743    251743
Name: index, Length: 251744, dtype: int64

In [ ]:
f_in=open("data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.beagle5.1.AGM.bgl.phased.aa",'r')
f_out=open("data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.beagle5.1.AGM.bgl.phased.QC.aa",'w')

ind_count=None
line_cnt=0

while True:
    line_cnt+=1
    line=f_in.readline().strip()
    line_split=line.split(' ')
    if not line:
        break
    if ind_count is None:
        ind_count=int(len(line_split[2:])/2)
    else:
        assert ind_count==int(len(line_split[2:])/2)
    
    line_type=line_split[0]
    line_id=line_split[1]
    line_data=line_split[2:]
    line_data=np.array(line_data)[iid_df_merged['index'].tolist()]
    
    
    
    if line_id in removed_aaformat_nodup:
        line_data_unique=np.unique(line_data)
        if len(line_data_unique)==2:
            print("removed all",line_data_unique,"from",line_id)
            continue
        elif len(line_data_unique)>2:
            total_removed=0
            #line_data_array=np.array(line_data)
            for removed_match in removed[removed.str.contains(line_id+'_')]:
                match_cnt=(line_data==removed_match[-1]).sum()
                total_removed+=match_cnt
                line_data[line_data==removed_match[-1]]='NA'
                print("removed",removed_match[-1],"from",line_id,"count:",match_cnt)
            if total_removed==0:
                raise
            f_out.write(' '.join([line_type,line_id]+line_data.tolist())+'\n')
            #f_out.write(' '.join([line_type,line_id]+line_data))
        else:
            raise
    else:
        f_out.write(' '.join([line_type,line_id]+line_data.tolist())+'\n')
        
f_in.close()
f_out.close()

removed all ['a' 'p'] from HLA_A*01:03
removed all ['a' 'p'] from HLA_A*02:02
removed all ['a' 'p'] from HLA_A*02:03
removed all ['a' 'p'] from HLA_A*02:05
removed all ['a' 'p'] from HLA_A*02:09
removed all ['a' 'p'] from HLA_A*02:10
removed all ['a' 'p'] from HLA_A*02:11
removed all ['a' 'p'] from HLA_A*03:02
removed all ['a' 'p'] from HLA_A*11:02
removed all ['a' 'p'] from HLA_A*11:03
removed all ['a' 'p'] from HLA_A*11:12
removed all ['a' 'p'] from HLA_A*11:77
removed all ['a' 'p'] from HLA_A*23:01
removed all ['a' 'p'] from HLA_A*24:03
removed all ['a' 'p'] from HLA_A*24:04
removed all ['a' 'p'] from HLA_A*24:07
removed all ['a' 'p'] from HLA_A*24:08
removed all ['a' 'p'] from HLA_A*24:10
removed all ['a' 'p'] from HLA_A*24:20
removed all ['a' 'p'] from HLA_A*25:01
removed all ['a' 'p'] from HLA_A*26:03
removed all ['a' 'p'] from HLA_A*29:01
removed all ['a' 'p'] from HLA_A*29:02
removed all ['a' 'p'] from HLA_A*31:02
removed all ['a' 'p'] from HLA_A*32:01
removed all ['a' 'p'] fro

# Merge

In [23]:
!echo 'data/genotype/0_1000G/KCHIP_HLA_6_28Mb_35Mb_20200113_QC.newfilter.nooverlap' > data/genotype/4_merge/merge_list.tsv

In [24]:
command='plink --bfile data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC.noaa \
--merge-list data/genotype/4_merge/merge_list.tsv \
--make-bed \
--out KCHIP_HLA_SNP_1000G_merged'
command

'plink --bfile data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC.noaa --merge-list data/genotype/4_merge/merge_list.tsv --make-bed --out KCHIP_HLA_SNP_1000G_merged'